In [1]:
#!pip install SPARQLWrapper

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import csv

In [3]:
LMDB_PREFIX = "<https://triplydb.com/Triply/linkedmdb/vocab/>"
SPARQL_ENDPOINT = "https://api.triplydb.com/datasets/Triply/linkedmdb/services/linkedmdb/sparql"
PROPERTY_NAMES = ["actor", "director", "genre", "cinematographer", "producer", "editor", "writer"] # must be from lmdb namespace

In [4]:
def get_movie_properties(movie):

    select_line = (" ").join(["?"+prop for prop in PROPERTY_NAMES])
    where_clause_lines = [f"Optional{{{movie} lmdb:{prop} ?{prop} . }}" for prop in PROPERTY_NAMES]
    where_clause = ("\n").join(where_clause_lines)

    query_str = f"""
    PREFIX lmdb: {LMDB_PREFIX}

    SELECT {select_line}
    WHERE {{
        {where_clause}
    }}
    """

    sparql = SPARQLWrapper(SPARQL_ENDPOINT)
    sparql.setQuery(query_str)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    properties = {prop: set() for prop in PROPERTY_NAMES}

    for result in results["results"]["bindings"]:
        for prop in PROPERTY_NAMES:
            if prop in result:
                properties[prop].add(result[prop]["value"])

    return properties

In [5]:
def get_movie_recommendations(properties, movie):    
    property_lines = []
    for prop, values in properties.items():
        for i, value in enumerate(values):
            property_lines.append(f"""
          {{
            ?movie lmdb:{prop} <{value}> .
            BIND("{prop}{i}" as ?matchedProperty)
          }}""")
    property_lines_combined = ("\nUNION\n").join(property_lines)

    query_str = f"""
    PREFIX lmdb: {LMDB_PREFIX}

    SELECT ?movie ?title ?year (GROUP_CONCAT(DISTINCT ?directorName; SEPARATOR=", ") AS ?directors) (COUNT(DISTINCT ?matchedProperty) as ?score)
    WHERE {{
      ?movie a lmdb:Film .
      ?movie <http://purl.org/dc/terms/title> ?title .
      ?base_movie <http://purl.org/dc/terms/title> ?base_title
      {property_lines_combined}
      ?movie lmdb:director ?director .
      ?director lmdb:director_name ?directorName .
      ?movie <http://purl.org/dc/terms/date> ?date .
      BIND(SUBSTR(?date, 1, 4) AS ?year) .
      FILTER(?base_movie = {movie} && ?title != ?base_title)
    }}
    GROUP BY ?movie ?title ?year ?directors
    ORDER BY DESC(?score)
    LIMIT 12
    """

    sparql = SPARQLWrapper(SPARQL_ENDPOINT)
    sparql.setQuery(query_str)
    sparql.setMethod('POST')
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    recommendations = []
    for result in results["results"]["bindings"]:
        movie_id = result["movie"]["value"].split('/')[-1]
        movie_title = result["title"]["value"]
        movie_director = result["directors"]["value"]
        movie_year = result["year"]["value"]
        score = result["score"]["value"]
        recommendations.append({"movie_id": movie_id, "movie_title": movie_title, "movie_director": movie_director, "movie_year": movie_year, "score": score})

    return recommendations

In [6]:
def recommend_movies(movie_id):
    movie = f"<https://triplydb.com/Triply/linkedmdb/id/film/{movie_id}>"
    properties = get_movie_properties(movie)
    similar_movies = get_movie_recommendations(properties, movie)
    return similar_movies
        
def print_movies(movie_list):
    print("Similar Movies:")
    for movie in movie_list:
        print(f"Movie ID: {movie['movie_id']}, Movie Title: {movie['movie_title']}, Score: {movie['score']}")

In [7]:
similar_movies = recommend_movies(72)
print_movies(similar_movies)

Similar Movies:
Movie ID: 33, Movie Title: Terminator 2: Judgment Day, Score: 8
Movie ID: 15605, Movie Title: Revolutionary Road, Score: 7
Movie ID: 6220, Movie Title: Avatar, Score: 7
Movie ID: 38324, Movie Title: True Lies, Score: 6
Movie ID: 556, Movie Title: Aliens, Score: 6
Movie ID: 2235, Movie Title: Strange Days, Score: 5
Movie ID: 3116, Movie Title: Poetic Justice, Score: 4
Movie ID: 1033, Movie Title: Tombstone, Score: 4
Movie ID: 3604, Movie Title: Little Children, Score: 4
Movie ID: 3296, Movie Title: T2 3-D: Battle Across Time, Score: 4
Movie ID: 37248, Movie Title: True Lies, Score: 4
Movie ID: 4673, Movie Title: Expedition: Bismarck, Score: 4


In [8]:
movie_list = []

with open('data/available_src_movies.csv') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        movie_list.append(row[0])

In [9]:
## Getting Movie IDs
## Problem: Some movies and directors have multiple ids.
## For movies, they often have one id >= 38,000 and one < 38,000
## Tentative solution: Only consider ids < 38,000
def get_movie_id(movie_title):
    query_str = f"""
    PREFIX lmdb: {LMDB_PREFIX}

    SELECT ?movie
    WHERE {{
      ?movie a lmdb:Film .
      ?movie <http://purl.org/dc/terms/title> "{movie_title}" .
      ?movie <http://purl.org/dc/terms/date> ?date .
      BIND(SUBSTR(?date, 1, 4) AS ?year) .
      FILTER NOT EXISTS {{
        ?otherMovie a lmdb:Film .
        ?otherMovie <http://purl.org/dc/terms/title> "{movie_title}" .
        ?otherMovie <http://purl.org/dc/terms/date> ?otherDate .
        BIND(SUBSTR(?otherDate, 1, 4) AS ?otherYear) .
        FILTER (?otherYear = ?year && ?otherMovie < ?movie)
        }}
    }}
    """
    
    sparql = SPARQLWrapper(SPARQL_ENDPOINT)
    sparql.setQuery(query_str)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    movie_ids = []
    for result in results["results"]["bindings"]:
        movie_id = result["movie"]["value"].split('/')[-1]
        movie_ids.append(movie_id)

    return movie_ids

In [10]:
movie_id_dic = {}

for movie in movie_list:
    found_ids = get_movie_id(movie)
    if (len(found_ids) == 1):
        movie_id_dic[movie] = found_ids[0]
    else:
        print(f"Manual correction required for '{movie}' with ids: {found_ids}.")
        movie_id_dic[movie] = found_ids

Manual correction required for 'Rebecca' with ids: ['11611', '2141'].
Manual correction required for 'Ben-Hur' with ids: ['2243', '2245'].
Manual correction required for 'The Great Escape' with ids: ['38824', '75494'].
Manual correction required for 'Bicycle Thieves' with ids: ['26568', '86895'].
Manual correction required for 'Heat' with ids: ['74985', '53', '9200', '76365', '77208'].
Manual correction required for 'High and Low' with ids: ['29381', '94309'].


In [11]:
## Manual correction
movie_id_dic["Rebecca"] = '2141'
movie_id_dic["Ben-Hur"] = '2245'
movie_id_dic["The Great Escape"] = '38824'
movie_id_dic["Bicycle Thieves"] = '26568'
movie_id_dic["Heat"] = '53'
movie_id_dic["High and Low"] = '29381'

In [12]:
## Create recommendations and write in csv file

with open('data/available_src_movies_recs.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    for movie_title, movie_id in movie_id_dic.items():
        recs = recommend_movies(movie_id)
        line = [movie_title]
        for rec in recs:
            line.append(rec['movie_title'])
        writer.writerow(line)
    print("Finished")

Finished
